In [1]:
from tm2py.acceptance import Acceptance
import numpy as np
import pandas as pd
import geopandas as gpd
import os

In [2]:
scenario_config_file = "../examples/scenario_config.toml"
model_config_file = "../examples/model_config.toml"
observed_config_file = "../examples/observed_data.toml"

In [3]:
a = Acceptance(scenario_config_file, model_config_file, observed_config_file)

In [4]:
a.make_acceptance(make_transit=False, make_roadway=False, make_other=True)